In [25]:
import pandas as pd
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

cred = credentials.Certificate("./key.json")
app = firebase_admin.initialize_app(cred)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [32]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [176]:
patients = pd.read_csv('patients.csv')
print(patients.columns[:24])
patients = patients.drop('Unnamed: 0', axis=1)
# patients.rename(columns={patients.columns})
patients.columns = map(str.lower, patients.columns)
patients['name'] = patients['first'].str.replace('\d+', '', regex=True) +' ' + patients['last'].str.replace('\d+', '', regex=True)
patients = patients.drop_duplicates(subset='name')
patients = patients.drop(['first', 'last'], axis=1)
patients = patients.rename(columns={'gender': 'sex'})

patients = patients.reset_index()

Index(['Unnamed: 0', 'Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS',
       'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')


In [174]:

pedi_index = []
family_index=[]
for i in patients.id.keys():
    if int(patients['birthdate'][i][:4]) > 2003:
        pedi_index.append(i)
    else:
        family_index.append(i)
len(family_index)

971

In [172]:
allergy_index = patients[patients['allergies'].notna()].index
allergy_index

Int64Index([  13,   20,   22,   26,   32,   34,   40,   45,   62,   80,
            ...
            1080, 1087, 1101, 1106, 1113, 1115, 1126, 1128, 1144, 1149],
           dtype='int64', length=141)

In [138]:
db = firestore.client()

def addData(s, doctor, patient_id): 
        mode_ref = db.collection(u'Professionals').document(doctor).collection(u'Patients').document(patient_id)
#         for k, v in s.iteritems():
        mode_ref.set(s.to_dict(),merge=True)



In [177]:
family = u'QM7V3Ud0gbOK907UD2rMnMQiGCN2'
health_insurance = u'Hn6SZn6uj4RA7k9Owk8KHu12cRH2'
pediatrician = u'yyRs0hOgEFXFJVe5ugZWystm1uh1'
allerginist = u'5yGQKp4Fk4UY0oLkc9e5ylczmYp2'

def profData(prof, indexes):
    for i in range(500):
        series = patients.loc[i, patients.iloc[i,:].notna().values][1:24]
        addData(series, prof, patients.loc[i, patients.iloc[i,:].notna().values][1])
profData(health_insurance, None)
